# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

In [4]:
chain = prompt | model

In [5]:
chain.invoke({'assunto': 'gatinhos'})

AIMessage(content='"Os gatinhos são seres adoráveis e fofinhos que enchem nossos corações de amor e carinho."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 19, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-68c61204-ceae-4216-8474-fd9521c1cf62-0', usage_metadata={'input_tokens': 19, 'output_tokens': 30, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Adicinando mais elementos a chain

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [9]:
chain.invoke({'assunto': 'gatinhos'})

'Os gatinhos são seres encantadores que alegram nossos dias com sua fofura e travessuras.'

### ATENÇÃO! A ordem importa

In [ ]:
chain = model | prompt | output_parser
chain.invoke({'assunto': 'gatinhos'})

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [11]:
input = {'assunto': 'gatinhos'}

In [12]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre o seguinte: gatinhos', additional_kwargs={}, response_metadata={})])

In [13]:
resposta = model.invoke(prompt_formatado)
resposta

AIMessage(content='Gatinhos são seres adoráveis que enchem nossas vidas de alegria e fofura.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-83a86ea6-0006-4efc-83be-3a0cdde01835-0', usage_metadata={'input_tokens': 19, 'output_tokens': 26, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
output_parser.invoke(resposta)

'Gatinhos são seres adoráveis que enchem nossas vidas de alegria e fofura.'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [ ]:
print(prompt.input_schema.schema_json(indent=4))

In [ ]:
chain = prompt | model | output_parser
print(chain.input_schema.schema_json(indent=4))

> Atenção!
O método schema_json será removido nas próximas versões do LangChain! É recomendado a utilização do método model_json_schema ao utilizar versões mais recents, da seguinte forma:

In [17]:
chain = prompt | model | output_parser
print(chain.input_schema.model_json_schema())

{'properties': {'assunto': {'title': 'Assunto', 'type': 'string'}}, 'required': ['assunto'], 'title': 'PromptInput', 'type': 'object'}


### Como eu teria feito com chains clássicas?

In [18]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")
output_parser = StrOutputParser()

In [19]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})

C:\Users\edinocencio\AppData\Local\Temp\ipykernel_61032\1936627178.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'assunto': 'gatinhos',
 'text': 'Gatinhos são seres adoráveis que conquistam nossos corações com seu jeito meigo e brincalhão.'}